## Generate templates

If you have already generated templates, move to `Generate recordings` section.

In [1]:
import MEArec as mr
import numpy as np
import matplotlib.pylab as plt
from pprint import pprint
%matplotlib notebook

In [2]:
templates_params = mr.get_default_templates_params()
cell_models_folder = mr.get_default_cell_models_folder()

In [3]:
print(cell_models_folder)

/Users/abuccino/.config/mearec/1.7.0/cell_models/bbp


In [4]:
pprint(templates_params)

{'check_for_drift_amp': False,
 'cut_out': [2, 5],
 'delay': 10,
 'drift_steps': 30,
 'drift_within_bounds': False,
 'drift_xlim': [-10, 10],
 'drift_ylim': [-10, 10],
 'drift_zlim': [30, 80],
 'drifting': False,
 'dt': 0.03125,
 'max_drift': 100,
 'max_iterations': 1000,
 'min_amp': 30,
 'min_drift': 30,
 'n': 50,
 'ncontacts': 10,
 'offset': 0,
 'overhang': 30,
 'probe': 'Neuronexus-32',
 'rot': 'physrot',
 'seed': None,
 'sim_time': 1,
 'target_spikes': [3, 50],
 'timeout': None,
 'weights': [0.25, 1.75],
 'xlim': [10, 80],
 'ylim': None,
 'zlim': None}


In [5]:
templates_params['probe'] = 'Neuropixels-384'
templates_params['n'] = 300

In [ ]:
tempgen = mr.gen_templates?

In [ ]:
tempgen = mr.gen_templates(cell_models_folder=cell_models_folder, params=templates_params, verbose=True,
                           n_jobs=4)

Running with 4 jobs


In [ ]:
# extract template amplitudes
amps = []
for t in tempgen.templates:
    amps.append(np.abs(np.min(t)))
amps = np.array(amps)

In [ ]:
_ = plt.hist(amps, bins=100)

In [ ]:
# save templates
templates_filename = 'templates_300_neuropixels.h5'
mr.save_template_generator(tempgen=tempgen, filename=templates_filename)

In [ ]:
# plot some templates
template_ids = np.random.permutation(len(tempgen.templates))[:10]
ax = mr.plot_templates(tempgen, template_ids=[3001], 
                       single_axes=False)

## Generate recordings

In [ ]:
# load templates (if generated)
templates_filename = 'templates_300_neuropixels.h5'
tempgen = mr.load_templates(templates_filename)

In [ ]:
recording_params = mr.get_default_recordings_params()
pprint(recording_params)

In [ ]:
# Set parameters
recording_params['spiketrains']['n_exc'] = 200
recording_params['spiketrains']['n_inh'] = 50
recording_params['spiketrains']['duration'] = 600
recording_params['spiketrains']['seed'] = 0


recording_params['templates']['min_amp'] = 40
recording_params['templates']['max_amp'] = 300
recording_params['templates']['seed'] = 0

recording_params['recordings']['modulation'] = 'electrode'
recording_params['recordings']['noise_mode'] = 'uncorrelated'
recording_params['recordings']['noise_level'] = 10
# use chunk options
recording_params['recordings']['chunk_conv_duration'] = 20
recording_params['recordings']['chunk_noise_duration'] = 20
recording_params['recordings']['chunk_filter_duration'] = 20
recording_params['recordings']['seed'] = 0

In [ ]:
recgen = mr.gen_recordings(templates=templates_filename, params=recording_params, verbose=True)

In [ ]:
mr.plot_recordings(recgen, start_time=0, end_time=2)

In [ ]:
# plot selected templates amplitudes
amps = []
for t in recgen.templates:
    amps.append(np.abs(np.min(t)))
amps = np.array(amps)

In [ ]:
fig = plt.figure()
ax_amp = fig.add_subplot(111)

_ = ax_amp.hist(amps, bins=50)

In [ ]:
# save recording
mr.save_recording_generator(recgen, 'recording_neuropixels_600s_350cells.h5')